In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
# Activate mypy type checking black formatting and set exception reporting mode
%load_ext nb_mypy
%xmode Minimal

In [ ]:
import chess
import chess.pgn
import json
import multiprocessing
import nbimporter  # Activate import functionality for notebooks
import os
import re
import subprocess
import time
from datetime import datetime
from IPython.display import clear_output, display

# Implementierung eines Spieles
Dieses Notebook führt alle einzelnen Dateien und Klassen zusammen und implementiert den Spielablauf einer Partie. Die verwendeten AI-Versionen können dabei frei gewählt werden. Ein menschlicher Spieler ist ebenfalls möglich.

## Vorbereitung
Alle Notebooks wurden unter Python mit der Version `3.10.4` erstellt. Im Folgenden kann die Version überprüft werden


In [ ]:
# debug
from platform import python_version

print("Required Python version:", "3.10.4")
print("Your Python version:\t", python_version())

# Setup commands
# conda create -n chess python=3.10.4 -c conda-forge
# conda activate chess
# pip install -r src/requirements.txt
# conda install jupyter -c conda-forge
# jupyter notebook

# Import

Folgende Dateien werden in dieser Reihenfolge importiert:
- `AI-Base-Class.ipynb`: Die abstrakte Basisklasse für alle AI-Versionen, welche das Eröffnungs- und Endspiel implementiert.
- `Exercise01AI`: Die erste Version der AI, welche im Mittelspiel zufällige Züge auswählt.
- `Exercise02AI`: Die zweite Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und dem einfachen Materialwert auswählt.
- `Exercise03AI`: Die dritte Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und der `Simplified Evaluation Function` auswählt.
- `Exercise04AI`: Die vierte Version der AI, welche im Vergleich zur dritten Version mithilfe von Memoisierung beschleunigt wurde.
- `Exercise05AI`: Die fünfte Version der AI, welche die dritte Version um das Alpha-Beta-Pruning erweitert.
- `Exercise06AI`: Die sechste Version der AI, welche die fünfte Version erneut um Memoisierung erweitert.
- `Exercise07AI`: Die siebte Version der AI, welche die sechste Version um Progressive Deepening erweitert.
- `Exercise08AI`: Die achte Version der AI, welche die siebte Version um die Singular Value Extension erweitert.
- `Exercise09AI`: HIER FEHLT EINE BESCHREIBUNG
- `HumanPlayer`: Ein Spieler welcher durch einen Menschen gesteuert werden kann.
- `StockfishPlayer`: Ein Spieler, welcher durch die Stockfish Engine gesteuert wird.

Anschließend wird das Spiel durchgeführt und auf der Festplatte als `.pgn`-Datei gespeichert.

In [ ]:
from AIBaseClass import ChessAI, State
from Exercise01AI import Exercise01AI
from Exercise02AI import Exercise02AI
from Exercise03AI import Exercise03AI
from Exercise04AI import Exercise04AI
from Exercise05AI import Exercise05AI
from Exercise06AI import Exercise06AI
from Exercise07AI import Exercise07AI
from Exercise08AI import Exercise08AI
from Exercise09AI import Exercise09AI
from HumanPlayer import HumanPlayer
from StockfishPlayer import StockfishPlayer

## Speichern der Partie

Die folgende Funktion speichert eine gegebene Partie (repräsentiert durch ein gespieltes Board) auf der Festplatte ab. Hierbei werden sämtliche Züge, die Spielernamen und einige weitere Metriken gespeichert.

Bei der Funktion $\textrm{parse_player_name}$ handelt es sich um eine Hilfsfunktion, die aus dem Klassennamen und dem beim Spielstart angegebenen Spielernamen einen String erstellt und diesen zurückgibt.

In [ ]:
def parse_player_name(player: ChessAI):
    """Parse the type of the current player and its name into a single string."""
    return f"{type(player).__name__} - {player.name}"

Die Funktion $\textrm{save_game}$ ist für die eigentliche Erstellung der `.pgn`-Datei zuständig. Hierfür nimmt diese die folgenden Parameter an:

1. `board`: ein abgeschlossenes Spiel in Form eines Boards,
2. `stats`: eine Liste von Dictionaries mit Statistiken für jeden Halbzug des Spiels,
3. `player1`: der weiße Spieler in Form einer Schach-KI-Version sowie
4. `player2`: der schwarze Spieler in Form einer Schach-KI-Version,
5. `rep`: Kurzform für $\textrm{repetitions}$ in Form eines Strings, der die Nummer des aktuellen Spiels sowie die Gesamtanzahl an Wiederholungen beinhaltet,
6. `duration`: eine Fließkommazahl, die die Gesamtzeit des Spiels beschreibt und
7. `seed`: eine Ganzzahl, die den für den Zufallsgenerator genutzten Seed speichert.

In [ ]:
def save_game(
    board: chess.Board,
    stats: list[dict],
    player1: ChessAI,
    player2: ChessAI,
    rep: str,
    duration: float,
    seed: int,
) -> None:
    """Saves last game to 'games/YYYY-mm-dd_HH-MM-SS.pgn' (based on current time)."""
    game = chess.pgn.Game.from_board(board)
    outcome = board.outcome(claim_draw=True)

    try:
        username = os.getlogin()
    except:
        username = "undefined"

    # set pgn headers
    game.headers["Event"] = "Chess-AI game"
    game.headers["Site"] = username
    game.headers["Date"] = datetime.now().strftime("%d.%m.%Y")
    game.headers["Commit"] = subprocess.check_output(["git", "rev-parse", "HEAD"]).decode("ascii").strip()
    game.headers["Round"] = rep
    game.headers["White"] = parse_player_name(player1)
    game.headers["Black"] = parse_player_name(player2)
    game.headers["Duration"] = str(duration)
    game.headers["Seed"] = str(seed)
    game.headers["Outcome"] = str(outcome.termination) if outcome else "Unknown"
    game.headers["Depth"] = str(getattr(player1, "DEPTH", "x")) + "-" + str(getattr(player2, "DEPTH", "x"))

    # add transitions as comments to pgn file
    assert len(list(game.mainline())) <= len(stats), "Missing statistics!"
    for node, statistics in zip(game.mainline(), stats):
        node.comment = json.dumps(statistics, default=lambda x: str(x))[1:-1]

    # write game to pgn file
    filename = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")
    with open(f"../games/{filename}.pgn", "w") as gamefile:
        # add a linebreak after each turn
        gamefile.write(re.sub(r" ([1-9][0-9]*)\.[^0-9]", r"\n\1.", str(game)))
        # add a newline to the end of the file
        gamefile.write("\n")

## Spielablauf (einzelnes Spiel)
Hier wird ein einzelnes neues Spiel initialisiert und anschließend gestartet. Die Funktion führt dabei nacheinander die Züge für die einzelnen Spieler aus und überwacht den aktuellen Spielzustand (State). Sobald ein Spieler den Endzustand erreicht hat, wird das Spiel beendet. Zusätzlich werden aktuelle Statistiken ausgegeben.


In [ ]:
def run_single_game(
    player1: ChessAI, player2: ChessAI, start_time: float
) -> tuple[chess.Board, list[dict]]:
    """Start a new chess game."""
    is_human_run = not bool(os.environ.get("CI", False))
    prefix = f"[{str(multiprocessing.current_process().pid)}]: " if not is_human_run else ""

    # Create a new board
    board = chess.Board()
    players = [player1, player2]

    # Show board before first move (for human player)
    if is_human_run:
        display(board)

    # Play game
    while True:
        for player in players:
            # Make next move
            t_start = time.time()
            player.make_turn(board)
            t_end = time.time()
            player.stats[-1]["time"] = t_end - t_start

            # Print board
            if is_human_run:
                clear_output(wait=True)
                display(board)

                # Calculate cache success statistics
                cache_tries = player.stats[-1].get("cache_tries", "N/A")
                if cache_tries == "N/A":
                    cache_hits = "N/A"
                    cache_percentage = "N/A"
                else:
                    cache_hits = player.stats[-1]["cache_hits"]
                    cache_percentage = "{:.2f}".format((100 / cache_tries) * cache_hits)

                print(f"Statistics {type(player).__name__} ({player.name}):")
                print(f"\tFen: {board.fen()}")
                print(f"\tState: {player.state}")
                print(f"\tCalculation time for last move: {'{:.2f}'.format(player.stats[-1]['time'])} s")
                if hasattr(player, "DEPTH"):
                    print(f"\tMax depth of last evaluation: {player.stats[-1].get('max_depth', 'N/A')}")
                    print(f"\tAverage depth of last evaluation: {player.stats[-1].get('avg_depth', 'N/A')}")
                print(f"\tCurrent board evaluation: {player.stats[-1].get('board_eval_after_move', 'N/A')}")
                print(f"\tMinimax board evaluation: {player.stats[-1].get('minimax_eval', 'N/A')}")
                print(f"\tSearched nodes: {player.stats[-1].get('nodes', 'N/A')}")
                print(f"\tCache tries: {cache_tries}")
                print(f"\tCache hits: {cache_hits} ({cache_percentage} %)")
            else:
                print(
                    f"{prefix}{type(player).__name__} ({player.state}) ==> '{board.fen()}'"
                    f"; cache_size_mb: {player.stats[-1].get('cache_size_mb', '-')}"
                )

            # Log state transitions
            if player.stats[-1]["state"] != player.stats[-2]["state"]:
                # Exit if game has finished
                if player.state == State.FINISHED:
                    print(f"{prefix}Finished with: {player.stats[-2]['state']} -> {State.FINISHED}")
                    outcome = board.outcome(claim_draw=True)
                    assert outcome, f"{prefix}Outcome is None in State.FINISHED and FEN {board.fen()}!"
                    if outcome.winner == chess.WHITE:
                        print(
                            f"{prefix}White won the match due to {str(outcome.termination)} with result {outcome.result()}!"
                        )
                    elif outcome.winner == chess.BLACK:
                        print(
                            f"{prefix}Black won the match due to {str(outcome.termination)} with result {outcome.result()}!"
                        )
                    else:
                        print(
                            f"{prefix}The game is a remis due to {str(outcome.termination)} with result {outcome.result()}!"
                        )
                    stats = [
                        item
                        for tpl in [tpl for tpl in zip(player1.stats, player2.stats)]
                        for item in tpl
                    ]
                    return board, stats[2:]

## Spielstart (mehrere Spiele)
Hier wird initial der Seed gesetzt und anschließend die gewünschte Anzahl an Spielen ausgeführt. Pro Spiel wird die Zeit gemessen. Alle Spiele werden anschließend abgespeichert.

In [ ]:
def run_games(player1: ChessAI, player2: ChessAI, repetitions: int, seed: int):
    is_human_run = not bool(os.environ.get("CI", False))
    prefix = f"[{str(multiprocessing.current_process().pid)}]: " if not is_human_run else ""
    seed = int(os.environ.get("CHESS_AI_SEED", seed))
    ChessAI.random.seed(seed)
    repetitions = int(os.environ.get("CHESS_AI_REPETITIONS", repetitions))

    for rep in range(repetitions):
        print(f"{prefix}Running game {rep+1} of {repetitions} with seed {seed}")
        print(f"{prefix}Player {type(player1).__name__} (white) vs {type(player2).__name__} (black)")
        t_start = time.time()
        board, transitions = run_single_game(player1, player2, t_start)
        t_end = time.time()
        rep_str = f"{rep + 1}/{repetitions}"
        save_game(board, transitions, player1, player2, rep_str, t_end - t_start, seed)
        player1.reset()
        player2.reset()

    print(prefix + f"Used seed {seed} with {repetitions} repetitions")

## Spielstart
Hier werden die Spieler definiert und die gewünschte Anzahl an Spielen gestartet.

**Hinweis**: Auch wenn nur ein Spiel gewünscht wird sollte die Funktion `run_games` verwendet werden. Nur in diesem Fall wird das resultierende Spiel automatisch gespeichert und der Seed ist definiert.

Beispielaufruf für ein einzelnes Spiel: `run_games(player1, player2, repetitions=1, seed=42)`  

Je nach Spieler sind verschiedene Zusatzargumente möglich die das Verhalten der Spieler anpassen.  
Beispiele:  
- `player1 = Exercise07AI(player_name="White", search_depth=5)`
- `player2 = StockfishPlayer(player_name="Black", elo=1500, time_limit=0.1)`

In [ ]:
%%time
# Create Players
player1 = StockfishPlayer(player_name="White")
player2 = Exercise09AI(player_name="Black", search_depth=2)

# Run game(s)
run_games(player1, player2, repetitions=1, seed=13)